In [1]:
from bs4 import BeautifulSoup
import urllib.request,urllib.error
import urllib.parse
import requests
import time
import random
from datetime import datetime
import os

/Users/jinanjiang/miniconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [17]:
failed_cases = []
i = 0

In [18]:

count=0
def askURL():#获取当前页的智能合约地址
    global i
    head={ #模拟浏览器头部信息
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
    }

    baseurl="https://etherscan.io/contractsVerified/" #获取智能合约地址页面 基地址
    while i <= 20:

        print(i)

        url=baseurl+str(i)#1-5页 加在基地址后面

        t=random.random()*1.0
        print(t)
        time.sleep(t)

        request = urllib.request.Request(url,headers=head,method="GET")#封装访问信息

        print(url)

        attempts = 0
        success = False
        while attempts < 5 and not success:
            try:
                response = urllib.request.urlopen(request, timeout=3.5)  # 访问网页，必须设置 访问时间超过多少，否则会被拒绝访问
                print("successfully get the page")
                success = True
            except:
                attempts += 1
                if attempts == 5:
                    print("attempt = 5,fail to get the ",i," page.")
                    continue



        # try:
        #     response = urllib.request.urlopen(request,timeout=3.5)#访问网页，必须设置 访问时间超过多少，否则会被拒绝访问
        #     print("ok")
        #
        # except:
        #      i = i-1
        #      continue
        html=response.read().decode("utf-8")#以gbk的方式解码，添加在列表里
        response.close()

        # print(html)
        Parse_html(html)#在每一个合约中抓取源代码
        i += 1



def Parse_html(html):
    
    cur_date = datetime.today().strftime('%Y-%m-%d')
    times = cur_date.split('-')
    assert len(times) == 3 and times[0] == '2023'
    
    if not os.path.exists('crawled_codes/' + cur_date):
        os.system('mkdir ' + 'crawled_codes/' + cur_date)
        
    if not os.path.exists('crawled_codes/' + cur_date + '_duplicates'):
        os.system('mkdir ' + 'crawled_codes/' + cur_date + '_duplicates')
        
    bs = BeautifulSoup(html,"html.parser")#解析每个html文件，
  #  print(bs)
    head = {  # 模拟浏览器头部信息，向豆瓣服务器发送消息
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
    }
  #  t_list=bs.find_all(class_="hash-tag text-truncate") #将每个页面中的25个合约地址获取到
   # t_list = bs.find_all("a",{'class':'me-1'})  # 将每个页面中的25个合约地址获取到
    t_list = bs.find_all(class_ = 'me-1')
   # pid = soup.findAll('a', {'class': 'sister'})
    result = []

    for i in t_list:
         if i.get('href')!= None:
               i=i.get('href')  # 对每项使用get函数取得tag属性值
               if(i.endswith('code')):
                    result.append(i)
    print(len(result))

    str1="https://etherscan.io/" #合约页面基地址/https://etherscan.io/0x0D18b9252a9Dc77d149930Abc49e3C2Dd7057e81#code
    str2="#code"#合约页面地址的最后部分，合约地址在str1、str2中间

  #  print(t_list)

    for item in result:
        global count #全局变量 文件名
        url=str1+item #拼全合约地址
        request = urllib.request.Request(url, headers=head, method="GET") #打包访问信息
        attempts = 0
        success = False
        while attempts < 3 and not success:
            try:
                response = urllib.request.urlopen(request,timeout=3.5) #访问合约页面
                success = True
            except:
                attempts += 1
                if attempts == 3:
                    continue

        # try:
        #
        #     response = urllib.request.urlopen(request,timeout=3.5) #访问合约页面
        # except:
        #     print("pass_"+url)
        #     pass
        #     continue
        try:
            contract = response.read().decode("utf-8")  #解析合约页面
        except:
            failed_cases.append(response)
            continue
        response.close()

        ds = BeautifulSoup(contract, "html.parser")#用html解析打开
        contract = ds.find_all(class_="js-sourcecopyarea editor") #定位页面中的合约信息
        print("contract:" + url)
        if len(contract)!=0:
            text=contract[0]#只取合约 去除标签
            result=text.get_text()# 转成string返回给result  因为write只能写string
            #filename = 'crawled_codes/contract_'+str(count) + '_sourceCode.sol'
            
            target_contract_address = url.split('address/')[1].split('#code')[0]
            assert target_contract_address.startswith('0x')

                
            filename = 'crawled_codes/' + cur_date + '/' + target_contract_address + '.txt'
            if os.path.exists(filename):
                cur_num = 0
                while True:
                    filename = 'crawled_codes/' + cur_date + '_duplicates' + '/' + target_contract_address + '_NUMBER' + str(cur_num) + '.txt'
                    if not os.path.exists(filename):
                        break
                    else:
                        cur_num += 1
           

            #assert not os.path.exists(filename)
            count=count+1
            
            with open(filename, 'w',encoding='utf-8') as file_object:
                 file_object.write(str(result))

        else:
            continue

In [108]:
i = 0

In [109]:
askURL()

0
0.6122154022975196
https://etherscan.io/contractsVerified/0
successfully get the page
25
contract:https://etherscan.io//address/0xA41161Fa10897d422aEF2E2d6AE3666Ac5b97dbc#code
contract:https://etherscan.io//address/0xD4133f1b8F4794341131Be42762916a7B51bdE8a#code
contract:https://etherscan.io//address/0x8761e0370f94f68db8eaa731f4fc581f6ad0bd68#code
contract:https://etherscan.io//address/0xa06CF5995bF01eCE7789976C7aCcC24701a7Cd02#code
contract:https://etherscan.io//address/0xf5c8D2b95b02f592442A8C4BbD1A0003B7a4A19f#code
contract:https://etherscan.io//address/0x179dC3fb0F2230094894317f307241A52CdB38Aa#code
contract:https://etherscan.io//address/0x966899c0d1e00c63aa635a2a19aa0d4ff7744bd6#code
contract:https://etherscan.io//address/0x0357c44Aa848e4C9C82aD3529e64E86bABa5971B#code
contract:https://etherscan.io//address/0x9574c8a1653717D2dA71269C17aC28e891E88067#code
contract:https://etherscan.io//address/0x2799396e7711299cf225e56fa7b8736b8da4e349#code
contract:https://etherscan.io//address/

contract:https://etherscan.io//address/0xb9b6774cFCEdF0a61BBf630E0DD78301415FA938#code
contract:https://etherscan.io//address/0xb3cF3C2396e894661DC6457dA0F5b74d9B6ba972#code
contract:https://etherscan.io//address/0xeb480090b413d08e553a5685f478e96b8e025df8#code
contract:https://etherscan.io//address/0xa655Aa809D1FE7a55e22425780bb676E8AE0A143#code
contract:https://etherscan.io//address/0xee4a1888fa81271cbbaee792e259fe3aa12cb43c#code
contract:https://etherscan.io//address/0xbb1b0ecdbeaef8daa3fa5bb8a016507a95743f0f#code
contract:https://etherscan.io//address/0xd9C8f70D7fc5E908F4598c6c4a5710fc5Ff780B1#code
contract:https://etherscan.io//address/0x911cd8acf8bacddbbd37664ba9010765dfae7c6f#code
contract:https://etherscan.io//address/0x37a152e0ab53a56da680c2ecd8b98003b03697b5#code
contract:https://etherscan.io//address/0x216DaF3d67D7a22D2873549175f05C01bFa20450#code
4
0.8342708350927517
https://etherscan.io/contractsVerified/4
successfully get the page
25
contract:https://etherscan.io//address/

contract:https://etherscan.io//address/0x8641d6b14695ccd57266ec03c416bd8b69315e91#code
contract:https://etherscan.io//address/0xb14a54971458a427f476a5b17e38ff56a7ffacc4#code
contract:https://etherscan.io//address/0x9cD5F7fb38eA8bd1a5e21D0b00128400cA9aFD9a#code
contract:https://etherscan.io//address/0xD4b777588c5A1F3a6734d07459cA801b9310c6Be#code
contract:https://etherscan.io//address/0x5aa5b575c3d885e91b9638d002851f81cfeaed55#code
contract:https://etherscan.io//address/0x03475e14de25908ed484eb99ad38d68cb6399807#code
contract:https://etherscan.io//address/0xd5dc24c9ef8a21e6d7caa603f9502e4a0dc7fe8a#code
contract:https://etherscan.io//address/0xff2bb61cb5cfbb6d4abf4c798b33d004bc00a7d6#code
contract:https://etherscan.io//address/0x10105641ff125a6062023Bca7C1A0B41B151301D#code
contract:https://etherscan.io//address/0xF412D1A0804690162d8b92c14ffa8Bf39ff2C876#code
contract:https://etherscan.io//address/0x862FE63EC1d383918F6472854751c62dd54A5dC3#code
contract:https://etherscan.io//address/0x54

contract:https://etherscan.io//address/0x18B954c648Bca3BEA56E9F68A28EBaF5bdcC6822#code
contract:https://etherscan.io//address/0x601698865d222639abb63a5ea1bde091af33116d#code
contract:https://etherscan.io//address/0x550614984b22e1a8deebff946583b3eed0f83f0b#code
11
0.4378722927910802
https://etherscan.io/contractsVerified/11
successfully get the page
25
contract:https://etherscan.io//address/0x31e87C9D5130EaCEF1da8a3932d010d1771eacf9#code
contract:https://etherscan.io//address/0x60cc54d16e9269624d77a69c0c8f827edae0e3ce#code
contract:https://etherscan.io//address/0x0a3a0056afae463c498bbc701fbea4b493abd902#code
contract:https://etherscan.io//address/0x501Ab24956AF6e43e5F5BEb75123cB96D1891C27#code
contract:https://etherscan.io//address/0x09FC83Be86dc771169eD8c92e05045Cc6491dc09#code
contract:https://etherscan.io//address/0x04D5eE1E3479ED597f9783a0B87c73eB0469b4ac#code
contract:https://etherscan.io//address/0xC5003D5F4877E78B530de70B046406748b348459#code
contract:https://etherscan.io//addres

contract:https://etherscan.io//address/0xa06a604184dc15d3bddc514a1c327957b7388128#code
contract:https://etherscan.io//address/0x6aa6e7342c52c62ee043735308b4ae718bb03896#code
contract:https://etherscan.io//address/0x9f98bef1a8f6b2c85f78eceef94ca5975e7c6065#code
contract:https://etherscan.io//address/0xaf0b8e23cd8c24290e2dcd8f4594f92ce7953172#code
contract:https://etherscan.io//address/0x72f82E63933E8C9f1F6753024ee43d73E4B8AEc0#code
contract:https://etherscan.io//address/0xccc184d5b756229ef6377439296d851c76cbd4ec#code
contract:https://etherscan.io//address/0x1272178b9d65e67420e450d06354caeaa53c50ca#code
contract:https://etherscan.io//address/0xa0c695033350c17f98405dc10604301bc0e18c10#code
contract:https://etherscan.io//address/0xb42ee30a61b08e5e9dbd8e54b11592b626c0571e#code
contract:https://etherscan.io//address/0x0871cd3fcee5b937e1b5ba88990d3272d5144eb1#code
contract:https://etherscan.io//address/0xa006aAEB781eBf2CE47285B3f601E131731679D8#code
contract:https://etherscan.io//address/0x05

contract:https://etherscan.io//address/0xd283a248020D2d2C0F89732BA5cB280A7A9c4872#code
contract:https://etherscan.io//address/0xb16ada27e4bc05bc7b113d0b0fd4962fe1ce2813#code
contract:https://etherscan.io//address/0xA9E7Bcb965dACf920d6c14F297432C3497cA19D4#code
contract:https://etherscan.io//address/0xC38480D067fD80f60Db05c28dd0C6130B38Bed37#code
contract:https://etherscan.io//address/0x29e95a16c277b97d04de93655ce7c08d1480d410#code
contract:https://etherscan.io//address/0x36aD0B0Da5A5B2876Cab5C38d95f733dE3D03fe4#code
contract:https://etherscan.io//address/0xa1583b2b7108bae3858cb5cb625479f63e1a2a10#code
contract:https://etherscan.io//address/0xc28bde7ca9c2d414165879d9643335d327373dd7#code
contract:https://etherscan.io//address/0x9274b07a2daec5dbb6efa61c28e49c1bc8f79abd#code
contract:https://etherscan.io//address/0xa948d6dbdf0b80adf9c49e9f2182ee4980345959#code
contract:https://etherscan.io//address/0x2dE7cC0453bA81a4d467bf30742fe5108C64Eb5C#code
contract:https://etherscan.io//address/0xfd

In [37]:
failed_cases

In [48]:
len('ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff')

64

In [46]:
115792089237316195423570985008687907853269984665640564039457584007913129639935 / 60 / 60 / 24 / 365

3.671743063080803e+69

In [45]:
(4294967295  / 60 / 60 / 24 / 365) + 1970

2106.192519501522

In [20]:
# cross reference codes

In [54]:
entire_c = '60806040526040516100109061007e565b604051809103906000f08015801561002c573d6000803e3d6000fd5b506000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555034801561007857600080fd5b5061008b565b61019b806107d483390190565b61073a8061009a6000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c80632078d35f146100515780632ca166761461006f578063343b278f1461009f578063d737d0c7146100cf575b600080fd5b6100596100ed565b604051610066919061049c565b60405180910390f35b6100896004803603810190610084919061059a565b610111565b604051610096919061063d565b60405180910390f35b6100b960048036038101906100b4919061059a565b610293565b6040516100c6919061063d565b60405180910390f35b6100d7610415565b6040516100e49190610667565b60405180910390f35b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b60008073ffffffffffffffffffffffffffffffffffffffff168673ffffffffffffffffffffffffffffffffffffffff16141580156101825750610152610415565b73ffffffffffffffffffffffffffffffffffffffff168673ffffffffffffffffffffffffffffffffffffffff1614155b156102855760008054905080156102835760008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1663c6171134306101d8610415565b6040518363ffffffff1660e01b81526004016101f5929190610682565b6020604051808303816000875af192505050801561023157506040513d601f19601f8201168201806040525081019061022e91906106d7565b60015b6102795761023d610415565b6040517fede71dcc0000000000000000000000000000000000000000000000000000000081526004016102709190610667565b60405180910390fd5b809250505061028a565b505b600190505b95945050505050565b60008073ffffffffffffffffffffffffffffffffffffffff168673ffffffffffffffffffffffffffffffffffffffff161415801561030457506102d4610415565b73ffffffffffffffffffffffffffffffffffffffff168673ffffffffffffffffffffffffffffffffffffffff1614155b156104075760008054905080156104055760008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1663c61711343061035a610415565b6040518363ffffffff1660e01b8152600401610377929190610682565b6020604051808303816000875af19250505080156103b357506040513d601f19601f820116820180604052508101906103b091906106d7565b60015b6103fb576103bf610415565b6040517fede71dcc0000000000000000000000000000000000000000000000000000000081526004016103f29190610667565b60405180910390fd5b809250505061040c565b505b600190505b95945050505050565b600030905090565b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b6000819050919050565b600061046261045d6104588461041d565b61043d565b61041d565b9050919050565b600061047482610447565b9050919050565b600061048682610469565b9050919050565b6104968161047b565b82525050565b60006020820190506104b1600083018461048d565b92915050565b600080fd5b600080fd5b60006104cc8261041d565b9050919050565b6104dc816104c1565b81146104e757600080fd5b50565b6000813590506104f9816104d3565b92915050565b6000819050919050565b610512816104ff565b811461051d57600080fd5b50565b60008135905061052f81610509565b92915050565b600080fd5b600080fd5b600080fd5b60008083601f84011261055a57610559610535565b5b8235905067ffffffffffffffff8111156105775761057661053a565b5b6020830191508360018202830111156105935761059261053f565b5b9250929050565b6000806000806000608086880312156105b6576105b56104b7565b5b60006105c4888289016104ea565b95505060206105d5888289016104ea565b94505060406105e688828901610520565b935050606086013567ffffffffffffffff811115610607576106066104bc565b5b61061388828901610544565b92509250509295509295909350565b60008115159050919050565b61063781610622565b82525050565b6000602082019050610652600083018461062e565b92915050565b610661816104c1565b82525050565b600060208201905061067c6000830184610658565b92915050565b60006040820190506106976000830185610658565b6106a46020830184610658565b9392505050565b6106b481610622565b81146106bf57600080fd5b50565b6000815190506106d1816106ab565b92915050565b6000602082840312156106ed576106ec6104b7565b5b60006106fb848285016106c2565b9150509291505056fea26469706673582212207137714206d318faaf719cf0b46f76051cfbbdcf6766327a7f4af8043cfdf48964736f6c63430008120033608060405234801561001057600080fd5b5061017b806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063c617113414610030575b600080fd5b61004a600480360381019061004591906100cf565b610060565b604051610057919061012a565b60405180910390f35b60006001905092915050565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b600061009c82610071565b9050919050565b6100ac81610091565b81146100b757600080fd5b50565b6000813590506100c9816100a3565b92915050565b600080604083850312156100e6576100e561006c565b5b60006100f4858286016100ba565b9250506020610105858286016100ba565b9150509250929050565b60008115159050919050565b6101248161010f565b82525050565b600060208201905061013f600083018461011b565b9291505056fea26469706673582212205d5cb8cd1aacc7165f6c64974bda95fd0256cb1cf00a55b8a1c1057b406eb6d764736f6c63430008120033'

In [55]:
sub_c = '608060405234801561001057600080fd5b5061017b806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063c617113414610030575b600080fd5b61004a600480360381019061004591906100cf565b610060565b604051610057919061012a565b60405180910390f35b60006001905092915050565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b600061009c82610071565b9050919050565b6100ac81610091565b81146100b757600080fd5b50565b6000813590506100c9816100a3565b92915050565b600080604083850312156100e6576100e561006c565b5b60006100f4858286016100ba565b9250506020610105858286016100ba565b9150509250929050565b60008115159050919050565b6101248161010f565b82525050565b600060208201905061013f600083018461011b565b9291505056fea26469706673582212209965fcda5149068c0fe93024d085f988eba8dca41ec0469d188cc068768a089f64736f6c63430008120033'

In [61]:
len(entire_c) / 2

2415.0

In [57]:
len(sub_c)

822

In [65]:
entire_c.index(sub_c[400:700])

4408

In [95]:
entire_c[4008:][740:750] == sub_c[740:750]

False

In [99]:
sub_c[740:] 

'fcda5149068c0fe93024d085f988eba8dca41ec0469d188cc068768a089f64736f6c63430008120033'

In [98]:
entire_c[4008:][740:] 

'b8cd1aacc7165f6c64974bda95fd0256cb1cf00a55b8a1c1057b406eb6d764736f6c63430008120033'

In [100]:
sub_c

'608060405234801561001057600080fd5b5061017b806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063c617113414610030575b600080fd5b61004a600480360381019061004591906100cf565b610060565b604051610057919061012a565b60405180910390f35b60006001905092915050565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b600061009c82610071565b9050919050565b6100ac81610091565b81146100b757600080fd5b50565b6000813590506100c9816100a3565b92915050565b600080604083850312156100e6576100e561006c565b5b60006100f4858286016100ba565b9250506020610105858286016100ba565b9150509250929050565b60008115159050919050565b6101248161010f565b82525050565b600060208201905061013f600083018461011b565b9291505056fea26469706673582212209965fcda5149068c0fe93024d085f988eba8dca41ec0469d188cc068768a089f64736f6c63430008120033'

In [102]:
entire_c

'60806040526040516100109061007e565b604051809103906000f08015801561002c573d6000803e3d6000fd5b506000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555034801561007857600080fd5b5061008b565b61019b806107d483390190565b61073a8061009a6000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c80632078d35f146100515780632ca166761461006f578063343b278f1461009f578063d737d0c7146100cf575b600080fd5b6100596100ed565b604051610066919061049c565b60405180910390f35b6100896004803603810190610084919061059a565b610111565b604051610096919061063d565b60405180910390f35b6100b960048036038101906100b4919061059a565b610293565b6040516100c6919061063d565b60405180910390f35b6100d7610415565b6040516100e49190610667565b60405180910390f35b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b60008073ffffffffffffffffffffffffffffffffffffffff168673ffffffffffffffffffffffffffffffffffffffff16141580156101825750610152610415565b73fffffffffff

In [106]:
a = '0x608060405234801561001057600080fd5b50610409806100206000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c806346ca626b1461003b57806357cc91e414610059575b600080fd5b61004361008d565b60405161005091906101ae565b60405180910390f35b610073600480360381019061006e919061022c565b610092565b6040516100849594939291906102a3565b60405180910390f35b603c81565b60008060008060006100ac606460c861012c610190610178565b9250600083116100f1576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016100e890610353565b60405180910390fd5b8673ffffffffffffffffffffffffffffffffffffffff168673ffffffffffffffffffffffffffffffffffffffff16101561013d5785945086935061013683603c610186565b915061016e565b869450859350826ec097ce7bc90715b34b9f100000000061015e91906103a2565b925061016b83603c610186565b91505b9295509295909350565b600060019050949350505050565b60006001905092915050565b60008160020b9050919050565b6101a881610192565b82525050565b60006020820190506101c3600083018461019f565b92915050565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b60006101f9826101ce565b9050919050565b610209816101ee565b811461021457600080fd5b50565b60008135905061022681610200565b92915050565b60008060408385031215610243576102426101c9565b5b600061025185828601610217565b925050602061026285828601610217565b9150509250929050565b610275816101ee565b82525050565b6000819050919050565b61028e8161027b565b82525050565b61029d816101ce565b82525050565b600060a0820190506102b8600083018861026c565b6102c5602083018761026c565b6102d26040830186610285565b6102df6060830185610294565b6102ec608083018461026c565b9695505050505050565b600082825260208201905092915050565b7f75696e742072617465207a65726f000000000000000000000000000000000000600082015250565b600061033d600e836102f6565b915061034882610307565b602082019050919050565b6000602082019050818103600083015261036c81610330565b9050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052601260045260246000fd5b60006103ad8261027b565b91506103b88361027b565b9250826103c8576103c7610373565b5b82820490509291505056fea264697066735822122087a7490de2326c28c262d6dfb03b5ba0ab40b623a6b7db9ce9afce643e9495d664736f6c63430008120033'

In [105]:
b = '0x608060405234801561001057600080fd5b50610409806100206000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c806346ca626b1461003b57806357cc91e414610059575b600080fd5b61004361008d565b60405161005091906101ae565b60405180910390f35b610073600480360381019061006e919061022c565b610092565b6040516100849594939291906102a3565b60405180910390f35b603c81565b60008060008060008594508693506100b2606460c861012c610190610178565b9250600083116100f7576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016100ee90610353565b60405180910390fd5b8673ffffffffffffffffffffffffffffffffffffffff168673ffffffffffffffffffffffffffffffffffffffff16101561013d5761013683603c610186565b915061016e565b869450859350826ec097ce7bc90715b34b9f100000000061015e91906103a2565b925061016b83603c610186565b91505b9295509295909350565b600060019050949350505050565b60006001905092915050565b60008160020b9050919050565b6101a881610192565b82525050565b60006020820190506101c3600083018461019f565b92915050565b600080fd5b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b60006101f9826101ce565b9050919050565b610209816101ee565b811461021457600080fd5b50565b60008135905061022681610200565b92915050565b60008060408385031215610243576102426101c9565b5b600061025185828601610217565b925050602061026285828601610217565b9150509250929050565b610275816101ee565b82525050565b6000819050919050565b61028e8161027b565b82525050565b61029d816101ce565b82525050565b600060a0820190506102b8600083018861026c565b6102c5602083018761026c565b6102d26040830186610285565b6102df6060830185610294565b6102ec608083018461026c565b9695505050505050565b600082825260208201905092915050565b7f75696e742072617465207a65726f000000000000000000000000000000000000600082015250565b600061033d600e836102f6565b915061034882610307565b602082019050919050565b6000602082019050818103600083015261036c81610330565b9050919050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052601260045260246000fd5b60006103ad8261027b565b91506103b88361027b565b9250826103c8576103c7610373565b5b82820490509291505056fea264697066735822122096efef23b95ce1c4d3930f35cf1a63faff168d2d01a1858e8b1ef25eb966ac1f64736f6c63430008120033'

In [107]:
a == b

False